In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
import random

n = 4

source = [i for i in range(n)]
# target = sorted(source, key=lambda _: random.random())
target = source
mapping = {k: v for k, v in zip(source, target)}

mapping[n] = n      # BOS
mapping[n+1] = n+1  # EOS

In [ ]:
def genseq(N):
    a = [n]
    for _ in range(N-2):
        a.append(random.randint(0, n-1))
    a.append(n+1)
    return a, [mapping[i] for i in a]

genseq(5)

In [ ]:
import torch
import torch.nn.functional as F

source_corpus = []
target_corpus = []

for _ in range(1):
    a, b = genseq(20)
    source_corpus.append(a)
    target_corpus.append(b)
    
source_corpus = torch.LongTensor(source_corpus)
target_corpus = torch.LongTensor(target_corpus)

In [ ]:
from model import Transformer
from torch.optim import AdamW

n = 4

model = Transformer(d_model=10, nhead=2, max_len=20, vocab_size=n+2,
                    encoder_layers=3, decoder_layers=3)

optim = AdamW(model.parameters(), lr=3e-4)

In [ ]:
from tqdm import tqdm

num_steps = 1000
batch_size = 16
eval_after = 100

for i in tqdm(range(num_steps)):
    model.train()
    samples = torch.LongTensor([random.randint(0, 0) for _ in range(batch_size)])
    input_ids = source_corpus[samples]
    output_ids = target_corpus[samples]
    loss, n = model.step(input_ids, output_ids)
    
    optim.zero_grad()
    (loss / n).backward()
    optim.step()
    
    if (i + 1) % eval_after == 0:
        print(f'most recent loss: {loss / n}')
        # model.eval()
        # tot_loss = 0
        # for j in range(10):
        #     input_ids = source_corpus[900+10*j:910+10*j]
        #     output_ids = target_corpus[900+10*j:910+10*j]
        #     loss, _ = model.step(input_ids, output_ids)
        #     tot_loss += loss
        # print(f'eval loss: {tot_loss / (100 * 19)}')

In [ ]:
samples = torch.LongTensor([random.randint(0, 900) for _ in range(1)])
input_ids = source_corpus[samples]
output_ids = target_corpus[samples]

print(input_ids, output_ids)

model.eval()
src, _ = model.get_src(input_ids)
tgt, _ = model.get_src(output_ids)

In [ ]:
logits, *_ = model.forward(src, tgt, None, None)
tok_probs = F.log_softmax(logits, dim=-1)

In [ ]:
print(tok_probs[:, :-1])

print(output_ids[:, 1:])

In [ ]:
from model import xent

xent(
    tok_probs[:, :-1],
    F.one_hot(output_ids[:, 1:], num_classes=6),
    ignore=-1
)

In [ ]:
F.softmax(logits, dim=-1).shape